In [33]:
import random
import time
import unittest
from unittest import TestCase, TestLoader, TextTestRunner
from enum import Enum, auto
from typing import List, Dict, Tuple, Optional, Set
import PySimpleGUI as sg
from dataclasses import dataclass

In [34]:
class TimeOfDay(Enum):
    MORNING = auto()  # Утро
    DAY = auto()      # День
    EVENING = auto()  # Вечер
    NIGHT = auto()    # Ночь

In [35]:
class World:
    def __init__(self, width: int, height: int):
        self.width = width
        self.height = height
        self.grid = [[None for _ in range(width)] for _ in range(height)]
        self.time_of_day = TimeOfDay.MORNING
        self.time_ticks = 0
        self.entities = []
        self.day_counter = 0

    def add_entity(self, entity, x: int, y: int):
        if not (0 <= x < self.width and 0 <= y < self.height):
            return False

        if self.grid[y][x] is not None:
            return False

        entity.x = x
        entity.y = y
        entity.world = self
        self.grid[y][x] = entity
        self.entities.append(entity)
        return True

    def remove_entity(self, entity):
        if entity in self.entities:
            self.entities.remove(entity)

        if (0 <= entity.x < self.width and
            0 <= entity.y < self.height and
            self.grid[entity.y][entity.x] == entity):
            self.grid[entity.y][entity.x] = None

    def get_neighbors(self, x: int, y: int) -> List[Tuple[int, int]]:
        neighbors = []
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                if dx == 0 and dy == 0:
                    continue
                nx, ny = x + dx, y + dy
                
                if 0 <= nx < self.width and 0 <= ny < self.height:
                    neighbors.append((nx, ny))
        return neighbors

    def update_time(self):
        self.time_ticks += 1
        if self.time_ticks % 6 == 0:
            prev_time = self.time_of_day
            self.time_of_day = {
                TimeOfDay.MORNING: TimeOfDay.DAY,
                TimeOfDay.DAY: TimeOfDay.EVENING,
                TimeOfDay.EVENING: TimeOfDay.NIGHT,
                TimeOfDay.NIGHT: TimeOfDay.MORNING
            }[self.time_of_day]
            
            if prev_time == TimeOfDay.NIGHT and self.time_of_day == TimeOfDay.MORNING:
                self.day_counter += 1

    def step(self):
        self.update_time()

        for entity in self.entities[:]:
            if isinstance(entity, Plant):
                entity.grow()
        
        for entity in random.sample(self.entities[:], len(self.entities[:])):
            if isinstance(entity, Animal):
                entity.update_behavior()  # Обновление поведения
                entity.move()            # Движение
                entity.eat()             # Питание
                entity.reproduce()       # Размножение
                entity.update_hunger()   # Обновление уровня голода

                if entity.energy <= 0:
                    self.remove_entity(entity)

    def display(self):
        time_names = {
            TimeOfDay.MORNING: "Утро",
            TimeOfDay.DAY: "День",
            TimeOfDay.EVENING: "Вечер",
            TimeOfDay.NIGHT: "Ночь"
        }
        print(f"День: {self.day_counter} Время: {time_names[self.time_of_day]} (Шаг: {self.time_ticks})")
        print(f"Существ: {len(self.entities)} (Растений: {sum(1 for e in self.entities if isinstance(e, Plant))}, "
              f"Травоядных: {sum(1 for e in self.entities if isinstance(e, Pauvre))}, "
              f"Хищников: {sum(1 for e in self.entities if isinstance(e, Malheureux))})")

        for y in range(self.height):
            row = []
            for x in range(self.width):
                entity = self.grid[y][x]
                if entity is None:
                    row.append('.')
                elif isinstance(entity, Lumiere):
                    row.append('L' if entity.active else 'l')
                elif isinstance(entity, Obscurite):
                    row.append('O' if entity.active else 'o')
                elif isinstance(entity, Demi):
                    row.append('D' if entity.active else 'd')
                elif isinstance(entity, Pauvre):
                    row.append('P' if not entity.sleeping else 'p')
                elif isinstance(entity, Malheureux):
                    row.append('M' if not entity.sleeping else 'm')
            print(' '.join(row))
        print()


In [36]:
class EcosystemMeta(type):
    
    _registry = {}
    
    def __new__(cls, name, bases, namespace):
        new_class = super().__new__(cls, name, bases, namespace)
        
        if name not in ['Plant', 'Animal']:
            cls._registry[name] = new_class
        
        if 'Plant' in [b.__name__ for b in bases]:
                new_class = cls._configure_plant(new_class, namespace)
        elif 'Animal' in [b.__name__ for b in bases]:
            new_class = cls._configure_animal(new_class, namespace)
            
        return new_class
    
    def __iter__(self):
        raise TypeError(f"'{self.__name__}' object is not iterable")    

    @classmethod
    def _configure_plant(cls, plant_class, namespace):
        
        # Получаем время роста из атрибутов класса
        growth_times = namespace.get('GROWTH_TIMES', [])
        
        # Динамически создаем метод can_grow_in_time
        def can_grow_in_time(self, time_of_day):
            return time_of_day in growth_times
            
        plant_class.can_grow_in_time = can_grow_in_time
        
        # Добавляем метод spread
        def spread(self):
            if not self.active:
                return
                
            neighbors = self.world.get_neighbors(self.x, self.y)
            random.shuffle(neighbors)
            
            for nx, ny in neighbors:
                target = self.world.grid[ny][nx]
                
                if target is None and random.random() < 0.2:
                    new_plant = self.__class__()
                    self.world.add_entity(new_plant, nx, ny)
                elif (isinstance(target, Plant) and 
                    target != self and 
                    self.active and not target.active and 
                    random.random() < 0.3):
                    target.health -= 20
                    if target.health <= 0:
                        self.world.remove_entity(target)
                        new_plant = self.__class__()
                        self.world.add_entity(new_plant, nx, ny)
        
        plant_class.spread = spread
        
        return plant_class
    
    @classmethod
    def _configure_animal(cls, animal_class, namespace):
        
        # Получаем параметры поведения из атрибутов класса
        food_types = namespace.get('FOOD_TYPES', [])
        sleep_times = namespace.get('SLEEP_TIMES', [])
        hunting_behavior = namespace.get('HUNTING_BEHAVIOR', {})
        
        # Динамически создаем метод get_food_types
        def get_food_types(self):
            return food_types
        
        # Динамически создаем метод modify_behavior
        def modify_behavior(self):
            self.sleeping = self.world.time_of_day in sleep_times
            if hasattr(self, 'pack'):
                self.update_pack()
            if hasattr(self, 'group'):
                self.update_group()
                
            # Применяем охотничье поведение если определено
            if hunting_behavior:
                self.hunting = (
                    not self.sleeping and 
                    self.hunger > hunting_behavior.get('min_hunger', 40)
                )

        def move(self):
            if self.sleeping or self.world is None:
                return

            possible_moves = [pos for pos in self.world.get_neighbors(self.x, self.y)
                            if self.world.grid[pos[1]][pos[0]] is None or 
                            (isinstance(self.world.grid[pos[1]][pos[0]], Plant) and 
                            random.random() < 0.3)]
            
            if possible_moves and random.random() < self.get_speed():
                nx, ny = random.choice(possible_moves)
                self.world.grid[self.y][self.x] = None
                self.x, self.y = nx, ny
                self.world.grid[ny][nx] = self
                self.energy -= 1
        
        def eat(self):
            if self.sleeping or self.hunger < 20 or self.world is None:
                return

            for nx, ny in self.world.get_neighbors(self.x, self.y):
                target = self.world.grid[ny][nx]
                if target is None:
                    continue
                    
                if any(isinstance(target, food_type) for food_type in self.get_food_types()):
                    nutrition = self.get_nutrition(target)
                    self.world.remove_entity(target)
                    self.hunger = max(0, self.hunger - nutrition)
                    self.energy += nutrition / 2
                    return
                
        def reproduce(self):
            if (self.energy < 80 or 
                self.hunger > 50 or 
                random.random() > 0.05 or 
                self.world is None):
                return

            for nx, ny in self.world.get_neighbors(self.x, self.y):
                target = self.world.grid[ny][nx]
                if (target is not None and 
                    isinstance(target, self.__class__) and 
                    target != self and 
                    target.energy > 70 and 
                    target.hunger < 50):
                    
                    for cx, cy in self.world.get_neighbors(self.x, self.y):
                        if self.world.grid[cy][cx] is None:
                            new_animal = self.__class__()
                            new_animal.energy = 60
                            if self.world.add_entity(new_animal, cx, cy):
                                self.energy -= 30
                                target.energy -= 30
                                return
        
        animal_class.get_food_types = get_food_types
        animal_class.modify_behavior = modify_behavior
        animal_class.move = move
        animal_class.eat = eat
        animal_class.reproduce = reproduce
        
        return animal_class
    
    

In [37]:
class Plant(metaclass=EcosystemMeta):
    def __init__(self):
        self.x = 0          # Координата X
        self.y = 0          # Координата Y
        self.world = None   # Ссылка на мир
        self.growth_rate = 1  # Скорость роста
        self.active = False  # Активен ли рост
        self.health = 100    # Здоровье растения
    def __iter__(self):
        raise TypeError(f"'{self.__class__.__name__}' object is not iterable")
        
    def __repr__(self):
            return f"<{self.__class__.__name__} at ({self.x}, {self.y})>"
    
    def grow(self):
        if self.world is None:
            return

        self.active = self.can_grow_in_time(self.world.time_of_day)
        
        if not self.active:
            return

        self.health = min(100, self.health + 5)
        self.spread()

    def __str__(self):
        return f"{self.__class__.__name__} в ({self.x}, {self.y})"


In [38]:
class Animal(metaclass=EcosystemMeta):
    def __iter__(self):
        raise TypeError(f"'{self.__name__}' object is not iterable")    

    def __init__(self):
        self.x = 0          # Координата X
        self.y = 0          # Координата Y
        self.world = None   # Ссылка на мир
        self.energy = 100   # Уровень энергии
        self.hunger = 0     # Уровень голода
        self.sleeping = False  # Спящий режим
        self.age = 0        # Возраст (в шагах симуляции)
        self.vision_radius = random.uniform(1.5, 3.0)

    def update_behavior(self):
        if self.world is None:
            return
        self.age += 1
        self.modify_behavior()  # Вызов специфичного поведения

    def get_speed(self) -> float:
        return max(0.1, 1.0 - (self.hunger / 200))

    def get_nutrition(self, food) -> int:
        if isinstance(food, Plant):
            return 30
        elif isinstance(food, Animal):
            return 50
        return 0

    def update_hunger(self):
        self.hunger += 2
        if self.hunger > 100:  # Голод снижает энергию
            self.energy -= 5    

    def __str__(self):
        return f"{self.__class__.__name__} в ({self.x}, {self.y})"


In [39]:

class Lumiere(Plant):
    GROWTH_TIMES = [TimeOfDay.DAY]

class Obscurite(Plant):
    GROWTH_TIMES = [TimeOfDay.NIGHT]

class Demi(Plant):
    GROWTH_TIMES = [TimeOfDay.EVENING, TimeOfDay.MORNING]


class Pauvre(Animal):
    FOOD_TYPES = (Lumiere,)  # Питается Lumiere
    SLEEP_TIMES = [TimeOfDay.NIGHT]  # Спит ночью

    def __init__(self):
        super().__init__()
        self.aggression = 0    # Уровень агрессии
        self.group = set()     # Группа особей
        self.group_size = 1    # Размер группы

    def update_group(self):
        self.group = {self}
        for dx in range(-3, 4):
            for dy in range(-3, 4):
                nx, ny = self.x + dx, self.y + dy
                if (0 <= nx < self.world.width and 
                    0 <= ny < self.world.height and 
                    isinstance(self.world.grid[ny][nx], Pauvre)):
                    self.group.add(self.world.grid[ny][nx])
        
        self.group_size = len(self.group)
        
        if self.group_size > 5 and random.random() < 0.1:
            self.split_group()

    def split_group(self):
        avg_x = sum(m.x for m in self.group) / len(self.group)
        avg_y = sum(m.y for m in self.group) / len(self.group)
        
        subgroup1 = set()
        subgroup2 = set()
        
        for member in self.group:
            if member.x < avg_x or (member.x == avg_x and member.y < avg_y):
                subgroup1.add(member)
            else:
                subgroup2.add(member)
        
        for member in subgroup1:
            dx = -1 if random.random() < 0.5 else 0
            dy = -1 if dx == 0 else 0
            nx, ny = member.x + dx, member.y + dy
            if (0 <= nx < self.world.width and 
                0 <= ny < self.world.height and 
                self.world.grid[ny][nx] is None):
                self.world.grid[member.y][member.x] = None
                member.x, member.y = nx, ny
                self.world.grid[ny][nx] = member

    def get_nutrition(self, food) -> int:
        if isinstance(food, Lumiere):
            if self.world.time_of_day == TimeOfDay.MORNING:
                return 50
            elif self.world.time_of_day == TimeOfDay.EVENING:
                return 20
            return 30
        return 0

class Malheureux(Animal):
    FOOD_TYPES = (Demi, Obscurite, Pauvre)  # Питается разными видами
    SLEEP_TIMES = [TimeOfDay.DAY, TimeOfDay.NIGHT]  # Спит днем и ночью
    HUNTING_BEHAVIOR = {
        'min_hunger': 40,  # Начинает охоту при голоде > 40
        'speed_boost': 1.2  # Увеличение скорости при охоте
    }
    def __init__(self):
        super().__init__()
        self.pack = set()    # Стая
        self.hunting = False # Режим охоты

    def update_pack(self):
        self.pack = {self}
        for dx in range(-2, 3):
            for dy in range(-2, 3):
                nx, ny = self.x + dx, self.y + dy
                if (0 <= nx < self.world.width and 
                    0 <= ny < self.world.height and 
                    isinstance(self.world.grid[ny][nx], Malheureux)):
                    self.pack.add(self.world.grid[ny][nx])
        
        if len(self.pack) > 3:
            for member in self.pack:
                member.aggression = True


    def get_speed(self) -> float:
        base_speed = super().get_speed()
        if self.hunting:
            return base_speed * 1.2  # Быстрее при охоте
        return base_speed * 0.8      # Медленнее в обычном режиме

In [40]:
class TestEcosystemMeta(TestCase):
    def test_plant_registration(self):
        self.assertIn('Lumiere', EcosystemMeta._registry)
        self.assertIn('Obscurite', EcosystemMeta._registry)
        self.assertIn('Demi', EcosystemMeta._registry)
    
    def test_animal_registration(self):
        self.assertIn('Pauvre', EcosystemMeta._registry)
        self.assertIn('Malheureux', EcosystemMeta._registry)
    
    def test_plant_methods(self):
        lumiere = Lumiere()
        self.assertTrue(hasattr(lumiere, 'can_grow_in_time'))
        self.assertTrue(hasattr(lumiere, 'spread'))
        
        self.assertTrue(lumiere.can_grow_in_time(TimeOfDay.DAY))
        self.assertFalse(lumiere.can_grow_in_time(TimeOfDay.NIGHT))
    
    def test_animal_methods(self):
        pauvre = Pauvre()
        self.assertTrue(hasattr(pauvre, 'get_food_types'))
        self.assertTrue(hasattr(pauvre, 'modify_behavior'))
        self.assertTrue(hasattr(pauvre, 'move'))
        self.assertTrue(hasattr(pauvre, 'eat'))
        
        self.assertEqual(pauvre.get_food_types(), (Lumiere,))

def run_tests():
    loader = TestLoader()
    suite = loader.loadTestsFromTestCase(TestEcosystemMeta)
    
    runner = TextTestRunner(verbosity=2)
    result = runner.run(suite)
    return result

test_result = run_tests()

test_animal_methods (__main__.TestEcosystemMeta.test_animal_methods) ... ok
test_animal_registration (__main__.TestEcosystemMeta.test_animal_registration) ... ok
test_plant_methods (__main__.TestEcosystemMeta.test_plant_methods) ... ok
test_plant_registration (__main__.TestEcosystemMeta.test_plant_registration) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.006s

OK


In [41]:
'''
def initialize_world(world: World, plant_density: float = 0.2, 
                    pauvre_count: int = 10, malheureux_count: int = 5):
    plant_classes = [Lumiere, Obscurite, Demi]  # Доступные типы растений
    
    num_plants = int(world.width * world.height * plant_density)
    
    for _ in range(num_plants):
        while True:
            x, y = random.randint(0, world.width-1), random.randint(0, world.height-1)
            if world.grid[y][x] is None:
                plant_class = random.choice(plant_classes)
                plant = plant_class()
                if world.add_entity(plant, x, y):
                    break
    
    for _ in range(pauvre_count):
        while True:
            x, y = random.randint(0, world.width-1), random.randint(0, world.height-1)
            if world.grid[y][x] is None:
                pauvre = Pauvre()
                if world.add_entity(pauvre, x, y):
                    break
    
    for _ in range(malheureux_count):
        while True:
            x, y = random.randint(0, world.width-1), random.randint(0, world.height-1)
            if world.grid[y][x] is None:
                malheureux = Malheureux()
                if world.add_entity(malheureux, x, y):
                    break

def run_simulation(width: int = 20, height: int = 20, 
                   plant_density: float = 0.2, 
                   pauvre_count: int = 10, malheureux_count: int = 5,
                   steps: int = 100, delay: float = 0.5):
    world = World(width, height)
    initialize_world(world, plant_density, pauvre_count, malheureux_count)
    
    try:
        for _ in range(steps):
            world.step()      # Шаг симуляции
            world.display()   # Отображение состояния
            time.sleep(delay) # Задержка для удобства наблюдения
    except KeyboardInterrupt:
        print("\nСимуляция остановлена пользователем.")
    except Exception as e:
        print(f"Ошибка: {e}")
    
    print("Симуляция завершена.")
    print(f"Итоговые показатели: День {world.day_counter}, Всего существ: {len(world.entities)}")
    print(f"Растений: {sum(1 for e in world.entities if isinstance(e, Plant))}")
    print(f"Травоядных: {sum(1 for e in world.entities if isinstance(e, Pauvre))}")
    print(f"Хищников: {sum(1 for e in world.entities if isinstance(e, Malheureux))}")

if __name__ == "__main__":
    run_simulation(
        width=20,            # Ширина мира
        height=20,           # Высота мира
        plant_density=0.2,   # Плотность растений (0.0 - 1.0)
        pauvre_count=15,     # Начальное количество травоядных
        malheureux_count=5,  # Начальное количество хищников
        steps=200,           # Количество шагов симуляции
        delay=0.3            # Задержка между шагами (сек)
    ) 
'''
1

1

In [42]:
@dataclass
class EntityConfig:
    color: str
    size: int = 10
    shape: str = 'circle'

COLORS = {
    'Lumiere': EntityConfig('#FFFF00', 8, 'rectangle'),
    'Obscurite': EntityConfig('#0000FF', 8, 'rectangle'),
    'Demi': EntityConfig('#808080', 8, 'rectangle'),
    'Pauvre': EntityConfig('#FFD700', 12, 'circle'),
    'Malheureux': EntityConfig('#800080', 15, 'circle')
}

In [ ]:
class WorldGUI:
    def __init__(self, world):
        self.world = world
        self.setup_gui()
        self.paused = False
        self.entity_shapes = {}
        
    def setup_gui(self):
        sg.theme('DarkGrey5')
        
        layout = [
            [sg.Text('Скорость:'), 
             sg.Slider(range=(1, 10), default_value=5, size=(20, 15), orientation='h', key='-SPEED-'),
             sg.Button('Пауза', key='-PAUSE-'),
             sg.Button('Сброс', key='-RESET-')],
             
            [sg.Graph(
                canvas_size=(800, 600),
                graph_bottom_left=(0, 0),
                graph_top_right=(self.world.width, self.world.height),
                key='-MAP-',
                enable_events=True,
                background_color='black'
            )],
            
            [sg.Multiline(
                size=(100, 8), 
                key='-STATS-', 
                disabled=True,
                background_color='#2E2E2E',
                text_color='white'
            )]
        ]
        
        self.window = sg.Window(
            'Симуляция Экосистемы', 
            layout, 
            finalize=True,
            resizable=True
        )
        
        self.graph = self.window['-MAP-']
        
    def draw_world(self):
        self.graph.Erase()
        self.entity_shapes = {}
        
        for y in range(self.world.height):
            for x in range(self.world.width):
                entity = self.world.grid[y][x]
                if entity:
                    self.draw_entity(x, y, entity)
        
        self.update_stats()
    
    def draw_entity(self, x, y, entity):
        color = COLORS.get(entity.__class__.__name__, 'white')
        
        if isinstance(entity, (Lumiere, Obscurite, Demi)):
            shape = self.graph.DrawRectangle(
                (x, y), (x + 1, y + 1),
                fill_color=color,
                line_color='white'
            )
        else:
            shape = self.graph.DrawCircle(
                (x + 0.5, y + 0.5), 0.4,
                fill_color=color,
                line_color='white'
            )
            
        self.entity_shapes[(x, y)] = shape
        
        # Рисуем радиус обзора для животных
        if hasattr(entity, 'vision_radius'):
            self.graph.DrawCircle(
                (x + 0.5, y + 0.5),
                entity.vision_radius,
                line_color=color,
                line_width=1
            )
    
    def update_stats(self):
        stats = {name: 0 for name in COLORS}
        vision_radii = []
        
        for entity in self.world.entities:
            name = entity.__class__.__name__
            stats[name] += 1
            if hasattr(entity, 'vision_radius'):
                vision_radii.append(entity.vision_radius)
        
        avg_vision = sum(vision_radii)/len(vision_radii) if vision_radii else 0
        
        stats_text = (
            f"День: {self.world.day_counter} | "
            f"Время: {self.world.time_of_day.name}\n"
            f"Растения: Lumiere={stats['Lumiere']}, Obscurite={stats['Obscurite']}, Demi={stats['Demi']}\n"
            f"Животные: Pauvre={stats['Pauvre']}, Malheureux={stats['Malheureux']}\n"
            f"Средний радиус обзора: {avg_vision:.2f}\n"
            f"Всего существ: {len(self.world.entities)}"
        )
        
        self.window['-STATS-'].update(stats_text)
    
    def run(self):
        while True:
            # Обновляем с учетом скорости
            speed = self.window['-SPEED-'].get()
            timeout = max(50, 200 - (speed * 20))
            
            event, values = self.window.read(timeout=timeout)
            
            if event in (sg.WIN_CLOSED, 'Exit'):
                break
                
            if event == '-PAUSE-':
                self.paused = not self.paused
                self.window['-PAUSE-'].update('Продолжить' if self.paused else 'Пауза')
                
            if event == '-RESET-':
                self.world = World(self.world.width, self.world.height)
                initialize_world(self.world)
                self.draw_world()
                
            if event == '-MAP-':
                mouse_pos = values['-MAP-']
                if mouse_pos != (None, None):
                    self.show_entity_info(mouse_pos)
            
            if not self.paused:
                self.world.step()
                self.draw_world()
        
        self.window.close()
    
    def show_entity_info(self, pos):
        x, y = int(pos[0]), int(pos[1])
        if 0 <= x < self.world.width and 0 <= y < self.world.height:
            entity = self.world.grid[y][x]
            if entity:
                info = str(entity)
                if hasattr(entity, 'vision_radius'):
                    info += f"\nРадиус обзора: {entity.vision_radius:.2f}"
                sg.popup_scrolled(info, title="Информация о существе")

In [44]:
def run_simulation_gui(width=40, height=30):
    world = World(width, height)
    initialize_world(world, plant_density=0.15, pauvre_count=20, malheureux_count=8)
    
    gui = WorldGUI(world)
    gui.run()

if __name__ == "__main__":
    run_simulation_gui()

AttributeError: module 'PySimpleGUI' has no attribute 'Text'